# Singularity

Documentation: https://sylabs.io/docs/



# Container technology comparison

<img src="https://tin6150.github.io/psg/fig/vm_vs_container.png">



[Container technology comparison](https://tin6150.github.io/psg/blogger_container_hpc.html)

## Build

### From pre-built images

Build command can only be performed by root or sudoer

Command syntax:  
```bash
$ {sudo} singularity build {image_name} {image_uri or definition file}
```


Examples:
```bash
$ sudo singularity build ubuntu.sif library://ubuntu
$ sudo singularity build lolcow.sif docker://godlovedc/lolcow
```

Sylabs Container Library: https://cloud.sylabs.io/library  
DockerHub: https://hub.docker.com/  


### From definition file

lolcow.def
```bash
BootStrap: library
From: ubuntu:16.04

%post
    apt-get -y update
    apt-get -y install fortune cowsay lolcat

%environment
    export LC_ALL=C
    export PATH=/usr/games:$PATH

%runscript
    fortune | cowsay | lolcat

%labels
    Author GodloveD
```

Build `lolcow.sif` from the `lolcow.def` definition file.
```bash
$ sudo singularity build lolcow.sif lolcow.def
```

### About `--sandbox`

```bash
$ sudo singularity build --sandbox sandbox.simg library://centos:latest
$ ls -ld sandbox.simg/
dr-xr-xr-x. 18 root root 4096 May 19  2019 sandbox.simg/
```
Do you see that its a directory?  
You can also `cd` into it.

```bash
$ cd sandbox.simg/
$ ls
bin  boot  dev  environment  etc  home  lib  lib64  media  mnt  opt  proc  root  run  sbin  singularity  srv  sys  tmp  usr  var
```
Looks just like a regular old linux root directory!

## Interacting with images

### Shell

```bash
$ singularity shell lolcow.sif

Singularity lolcow.sif:~> whoami
username
Singularity lolcow.sif:~> id
uid=1000(username) gid=1000(username) groups=1000(username)
Singularity lolcow.sif:~> exit

$ sudo singularity shell lolcow.sif
Singularity lolcow.sif:~> whoami
root
Singularity lolcow.sif:~> id
uid=0(root) gid=0(root) groups=0(root)
```

Use the `--writable` option to persist changes in the container.  

Readonly Example:
```bash
# Shell inside the container 
$ singularity shell sandbox.simg

# Try and make a file in the root directory
Singularity sandbox.simg:~> echo "Hello World" > /hello.txt
bash: /hello.txt: Read-only file system
```
Error. The file cannot be made.  

Give the `--writable` option to persist changes in the container.  
```bash
# shell back inside the container
# You probably need to shell as root unless you changed the permission of the container
$ sudo singularity shell --writable sandbox.simg

# Make the same file as last time, only it works this time.
Singularity sandbox.simg:~> echo "Hello World" > /hello.txt

# Exit the container and come back in
Singularity sandbox.simg:~> exit
$ singularity shell sandbox.simg

# check if the previously made file exists
Singularity sandbox.simg:~> cat /hello.txt

```

### Executing Commands

```bash
$ singularity exec lolcow.sif cowsay moo
```

### Running a container

```bash
$ singularity run lolcow_latest.sif
```

### Binding (`--bind` / `-B`)

By default Singularity bind mounts `/home/$USER`, `/tmp`, and `$PWD` into your container at runtime.